#### GROMACS for CHARMM-GUI
(See also: [`GROMACS_for_production.ipynb`](https://colab.research.google.com/github/bioinfkaustin/gromacs-on-colab/blob/main/GROMACS_for_production.ipynb).)

#### Documentation
**Before using this notebook, please click the *↳ cells hidden* button below to show the documentation.**

##### License

> This notebook as a work of software is licensed under the terms of the [AGPL-3.0](https://opensource.org/licenses/AGPL-3.0) or later.

##### About this software

> This notebook processes a **CHARMM-GUI system archive** (`.tgz`), producing a **GROMACS-ready folder for production runs**.
>
> A protein system prepared with the CHARMM-GUI **Solution Builder** or **Membrane Builder** must be provided.  
>
> The recommended **minimisation** and **equilibration** simulations are then run with **GROMACS**, which automatically utilises the GPU if one is allocated. The equilibrated system is saved for a later production simulation. 

##### Piecewise preparation of protein-ligand complexes

> *Optionally*, a docked ligand conformation prepared with the CHARMM-GUI **Ligand Reader** may be provided, in which case the two separate structures and topologies will be **merged into a protein-ligand complex**.
>
> To merge **multiple** cooperatively bound ligands, **multiple paths to archive files** are separated by the `|` keyword.

> $\color{orangered}{/\textbf{!}\backslash}$ These files **must** come from putting the **docking output** into **Ligand Reader**. $\color{orangered}{/\textbf{!}\backslash}$
>
> Preparing the ligand system using the docking output is the only way the coordinates of the docked pose can go into CHARMM-GUI and then into GROMACS.

> *e.g.*
> ```
> ligand_archives = "{GoogleDrive}/diazepam.tgz | {GoogleDrive}/GABA.tgz"
> ```
> The above example merges docked conformations of diazepam and GABA into a protein system.

#### Configuration

In [ ]:
import os
import re
import shutil
from functools import partial

#@markdown Provide the location of the `.tgz` from **Solution Builder** or **Membrane Builder**. (If you do not wish to use Google Drive, then in the left pane, go to *Files*, then *Upload*.)
protein_archive = "{GoogleDrive}/CHARMM-GUI/7FBF_FABPH.tgz" #@param {type: "string"}
# default: {GoogleDrive}/CHARMM-GUI/7FBF_FABPH.tgz

#@markdown Specify a new folder in which to save the equilibrated output -- the production simulation can then be run in this folder.
output_folder = "{GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid" #@param {type: "string"}
# default: {GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid

#@markdown If applicable, please see also the advanced settings below. **After filling in this form, run the notebook by clicking *Runtime -> Run all* in the toolbar.**

#@markdown \
#@markdown **Merging docked ligands into the system**
#@markdown 
#@markdown Optionally, docked ligand conformations may be added to the system, provided as `.tgz` archives from **Ligand Reader**.
ligand_archives = "{GoogleDrive}/CHARMM-GUI/7FBF_octanoic_acid.tgz" #@param {type: "string"}
# default: {GoogleDrive}/CHARMM-GUI/7FBF_octanoic_acid.tgz


#
# Google Drive
#

if not os.path.isdir("/content/drive/MyDrive") and any(s.startswith("{GoogleDrive}") for s in (protein_archive, output_folder, ligand_archives)):
  from google.colab import drive
  drive.mount("/content/drive")


#
# Methods for parsing and validation
#

def _partition(s):
  values = s.split("<>")
  if len(values) == 1:  return {"top": values[0], "bottom": values[0]}
  elif len(values) == 2:  return {"top": values[0], "bottom": values[1]}
  else:  raise ValueError(f"Error: expected top <> bottom, but got {len(values) - 2} extra <>: {s}")

def _multiple(s):
  return list(filter(None, s.split("|")))

def _path(s, exists=False, exts=None):
  if "{GoogleDrive}" in s and not s.startswith("{GoogleDrive}"):  raise ValueError(f"Error: {{GoogleDrive}} is a path prefix, but appears later: {s}")
  s = s.format(GoogleDrive="/content/drive/MyDrive")
  #     ^^^ raises KeyError if any {...} placeholder is present except {GoogleDrive}

  if exists  and not os.path.isfile(s):  raise FileNotFoundError(f"Error: file not found: {s}")
  if exts    and not any(s.endswith(ext) for ext in exts):  raise ValueError(f"Error: expecting file extension like {', '.join(exts)}, but got: {s}") 
  return os.path.abspath(s)

_extra_spaces = re.compile(r"(^ +|(?<=\|) +| +(?=\|)| +$)")
def parse(s, multiple=False, mandatory=False, path=False, exists=False, exts=None):
  # e.g. "  ...  " -> "..."
  s = _extra_spaces.sub("", s)
  # e.g. "a | b | c" -> ["a", "b", "c"]
  if multiple:
    s = _multiple(s)
  # error on e.g. ""
  if mandatory and (
    multiple         and not any(v for v in s)
    or not multiple  and not s
  ):
    raise ValueError("Error: mandatory setting without value")
  # e.g. {GoogleDrive}/archive.tgz -> /content/drive/MyDrive/archive.tgz
  if path:
    _path_preset = partial(_path, exists=exists, exts=exts)
    if multiple:  s = [_path_preset(v) for v in s]
    else:         s = _path_preset(s)
  return s


#
# Parse and validate the input values
#

# protein_archive

archive_exts = [".tgz", ".tar.gz"]
protein_archive = parse(protein_archive, mandatory=True, path=True, exists=True, exts=archive_exts)



# output_folder

output_folder = parse(output_folder, mandatory=True, path=True)

os.makedirs(output_folder, exist_ok=True)

if os.path.isfile(output_folder + "/conf.gro"):
  raise RuntimeError(f"Error: expecting empty folder, but found existing output: {output_folder}")


# ligand_archives

ligand_archives = parse(ligand_archives, multiple=True, path=True, exists=True, exts=archive_exts)

ligands_bash = "|".join(ligand_archives)


#
# Make sure that the notebook is in the start folder
#

if "START" not in os.environ or not os.environ["START"]:
  %env START={os.getcwd()}
else:
  %cd {os.environ["START"]}


#
# Use a clean scratch directory for the rest of the run
#

try:
  shutil.rmtree("scratch")
except FileNotFoundError:
  pass
os.makedirs("scratch")
%cd "scratch"

#### Input

In [ ]:
%%bash -s "$protein_archive"
protein_archive="$1"
#@markdown Extract the system from the protein archive. This should come from either **Solution Builder** or **Membrane Builder**.

if [[ -d "protein" ]]; then
  exit 0  # already extracted protein
fi

tar -xzf "${protein_archive}"

if ! compgen -G "charmm-gui-*/" > /dev/null; then
  echo "Error: not in CHARMM-GUI archive format: ${protein_archive}" 1>&2
  exit 1
fi
mv charmm-gui-*/ "protein"

In [ ]:
%%bash -s "$ligands_bash"
ligand_archives="$1"
#@markdown Extract the docked ligands from the ligand archives, if given. These files **must** come from putting the **docking output** into **Ligand Reader**.

if compgen -G "ligand_*/" > /dev/null; then
  exit 0  # already extracted ligand(s)
fi

i=1
IFS="|"
for ligand_archive in $ligand_archives; do
  tar -xzf "${ligand_archive}"

  if ! compgen -G "charmm-gui-*/" > /dev/null; then
    echo "Error: not in CHARMM-GUI archive format: ${ligand_archive}" 1>&2
    exit 1
  fi
  mv charmm-gui-*/ "ligand_${i}"
  i=$(($i + 1))
done

#### Installation

In [ ]:
#@markdown In the following cells, applications are preferentially downloaded from a **persistent cache** in your Google Drive. If not found there, they will be downloaded from the internet and compiled and/or installed instead.
#@markdown
#@markdown This cell sets up the cache folder.

if os.path.isdir("/content/drive/MyDrive"):
  storage = "/content/drive/MyDrive/gromacs-on-colab"
  os.makedirs(storage, exist_ok=True)
  %env STORAGE={storage}
else:
  %env STORAGE=/dev/null  # deliberately not a folder

In [ ]:
%%bash
#@markdown **GROMACS** is downloaded and compiled from source code. (This takes a while.)

if [[ -d "/usr/local/gromacs" ]]; then
  exit 0  # already installed
fi

gromacs_vers="2023" #@param {type: "string"}
cache_gromacs="${STORAGE}/gromacs-${gromacs_vers}.tar.gz"

if [[ -s "${cache_gromacs}" ]]; then
  tar -xzf "${cache_gromacs}" -C "/usr/local"
else
  wget -q "ftp://ftp.gromacs.org/gromacs/gromacs-${gromacs_vers}.tar.gz"
  if [[ ! -s "gromacs-${gromacs_vers}.tar.gz" ]]; then
    echo "Error: could not download: gromacs-${gromacs_vers}.tar.gz" >&2
    exit 1
  fi
  tar -xzf "gromacs-${gromacs_vers}.tar.gz"
  rm "gromacs-${gromacs_vers}.tar.gz"

  cd "gromacs-${gromacs_vers}"
  mkdir "build"
  cd "build"
  cmake .. -DGMX_BUILD_OWN_FFTW=ON -DGMX_GPU=CUDA
  make -j $(nproc)
  make install # -> /usr/local/gromacs

  if [[ -d "$(dirname "${cache_gromacs}")" ]]; then
    tar -czf "my_gromacs.tar.gz" -C "/usr/local" "gromacs"
    mv "my_gromacs.tar.gz" "${cache_gromacs}"
  fi
fi

In [ ]:
%%bash
#@markdown Set up a Miniconda environment with **Python** `<=3.8`, **Numpy** `<=1.23`, and **NetworkX** `=2.3`, for `cgenff_charmm2gmx.py`; and a second environment for **Biopython** and **Open Babel**.

if [[ -d "${START}/miniconda3" ]]; then
  exit 0  # already installed
fi

miniconda3_vers="py39_23.1.0-1" #@param {type: "string"}
wget -q "https://repo.anaconda.com/miniconda/Miniconda3-${miniconda3_vers}-Linux-x86_64.sh"
if [[ ! -s "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh" ]]; then
  echo "Error: could not download: Miniconda3-${miniconda3_vers}-Linux-x86_64.sh" >&2
  exit 1
fi
bash "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh" -b -p "${START}/miniconda3"
rm "Miniconda3-${miniconda3_vers}-Linux-x86_64.sh"

eval "$("$START/miniconda3/bin/conda" shell.bash hook)"

cache_miniconda3="${STORAGE}/Miniconda3-${miniconda3_vers}-Linux-x86_64_envs.tar.gz"

if [[ -s "${cache_miniconda3}" ]]; then
  tar -xzf "${cache_miniconda3}" -C "${START}/miniconda3"
else
  conda create --yes --name "charmm2gmx"
  conda activate "charmm2gmx"
  conda config --env --set channel_priority "strict"
  conda config --env --add channels "conda-forge"
  conda install -q --yes "python<=3.8" "networkx=2.3" "numpy<=1.23"
  
  conda create --yes --name "biopython"
  conda activate "biopython"
  conda config --env --set channel_priority "strict"
  conda config --env --add channels "conda-forge"
  conda install -q --yes "biopython" "openbabel"

  if [[ -d "$(dirname "${cache_miniconda3}")" ]]; then
    tar -czf "my_miniconda3_envs.tar.gz" -C "${START}/miniconda3" "envs"
    mv "my_miniconda3_envs.tar.gz" "${cache_miniconda3}"
  fi
fi

In [ ]:
%%bash
#@markdown The CHARMM36 forcefield is downloaded.

if [[ -d "${START}/charmm36.ff" ]]; then
  exit 0  # already installed
fi

charmm36_vers="jul2022" #@param {type: "string"}
cache_charmm36="${STORAGE}/charmm36-${charmm36_vers}.tar.gz"

if [[ -s "${cache_charmm36}" ]]; then
  tar -xzf "${cache_charmm36}" -C "${START}"
else
  wget -q -O "charmm36-${charmm36_vers}.ff.tgz" "https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/charmm36-${charmm36_vers}.ff.tgz"
  if [[ ! -s "charmm36-${charmm36_vers}.ff.tgz" ]]; then
    echo "Error: could not download: charmm36-${charmm36_vers}.ff.tgz" >&2
    exit 1
  fi
  tar -xzf "charmm36-${charmm36_vers}.ff.tgz"
  rm "charmm36-${charmm36_vers}.ff.tgz"
  
  mv "charmm36-${charmm36_vers}.ff" "${START}/charmm36.ff"

  if [[ -d "$(dirname "${cache_charmm36}")" ]]; then
    tar -czf "my_charmm36.tar.gz" -C "${START}" "charmm36.ff"
    mv "my_charmm36.tar.gz" "${cache_charmm36}"
  fi
fi

In [ ]:
%%bash
#@markdown The utility **`cgenff_charmm2gmx.py`** is installed.

if [[ -x "$START/miniconda3/envs/charmm2gmx/bin/cgenff_charmm2gmx.py" ]]; then
  exit 0  # already installed
fi

charmm2gmx_vers="py3_nx2" #@param {type: "string"}
cache_charmm2gmx="$STORAGE/cgenff_charmm2gmx_${charmm2gmx_vers}.tar.gz"

if [[ -s "${cache_charmm2gmx}" ]]; then
  tar -xzf "${cache_charmm2gmx}" -C "${START}/miniconda3/envs/charmm2gmx/bin"
else
  wget -q -O "cgenff_charmm2gmx.py" "https://mackerell.umaryland.edu/download.php?filename=CHARMM_ff_params_files/cgenff_charmm2gmx_${charmm2gmx_vers}.py"
  if [[ ! -s "cgenff_charmm2gmx.py" ]]; then
    echo "Error: could not download: cgenff_charmm2gmx_${charmm2gmx_vers}.py" >&2
    exit 1
  fi
  sed -i"" 's/\r//g' "cgenff_charmm2gmx.py"
  chmod +x "cgenff_charmm2gmx.py"
  mv "cgenff_charmm2gmx.py" "${START}/miniconda3/envs/charmm2gmx/bin/"

  if [[ -d "$(dirname "${cache_charmm2gmx}")" ]]; then
    tar -czf "my_charmm2gmx.tar.gz" -C "${START}/miniconda3/envs/charmm2gmx/bin" "cgenff_charmm2gmx.py"
    mv "my_charmm2gmx.tar.gz" "${cache_charmm2gmx}"
  fi
fi

#### Library

In [ ]:
#@markdown A class which allows for the parsing and limited editing of **GROMACS ".itp" topology files**.
#@markdown ```
#@markdown class Itp:
#@markdown   def __init__(self, filename): ...
#@markdown   ...
#@markdown ```

from collections import namedtuple
T = namedtuple("T", ["line", "comment"])

class Itp:
  def __init__(self, filename):
    self.blocks = [None]
    self.data = [[]]
    with open(filename) as f:
      self.__parse(f)

  def __parse(self, lines):
    columns = re.compile(r"\b(?=[ \t])")
    concat = None
    for l in lines:
      l = l.rstrip("\n") # newline
      l = l.replace("\t", "  ") # tabs are a valid kind of whitespace in .itp files, but this keeps things simple
      if l.endswith("\\"): # a line ending with a backslash runs on to the next line
        if concat is None:
          concat = l[:-1] + " "
        else:
          concat += l[:-1] + " "
        continue
      elif concat is not None:
        l = concat + l
        concat = None
      try:
        l, c = l.split(";") # comments
      except ValueError:
        c = None

      l_ = l.strip()
      if l_.startswith("["): # start of block?
        self.blocks.append(T(line=l, comment=c))
        self.data.append([])
      elif l_ and not l_.startswith("#"): # data line?
        s = columns.split(l)
        self.data[-1].append(T(line=s, comment=c))
      else: # comment or blank line or preprocessor directive
        self.data[-1].append(T(line=l, comment=c))

  def print(self, file=None):
    for block, data in zip(self.blocks, self.data):
      if block is not None:
        print(self._str(block), file=file)
      for t in data:
        print(self._str(t), file=file)

  def _str(self, t):
    l = t.line if isinstance(t.line, str) else "".join(t.line)
    if t.comment is not None:
      l += ";" + t.comment
    return l

  @classmethod
  def only(cls, data):
    return filter(lambda t: isinstance(t[0], list), data)

  #
  # Static variables -- these are helper data for various applications
  #

  # For each forcefield.itp / molecule.prm block, try to stick to the documented order where possible -- https://manual.gromacs.org/current/reference-manual/topologies/topology-file-formats.html
  directive_order_hint = [None, "defaults", "atomtypes", "bondtypes", "pairtypes", "angletypes", "dihedraltypes", "constrainttypes", "nonbond_params"]

  # For each molecule.itp block, this table gives the number of atom id columns an item within that block must have
  block_num_atom_cols = {
    "atoms":                  1,    "bonds":                  2,    "pairs":                     2,
    "pairs_nb":               2,    "angles":                 3,    "dihedrals":                 4,
    "exclusions":            -1,    "constraints":            2,    "settles":                   1,
    "virtual_sites1":         2,    "virtual_sites2":         3,    "virtual_sites3":            4,
    "virtual_sites4":         5,    "virtual_sitesn":         1,    "position_restraints":       1,
    "distance_restraints":    2,    "dihedral_restraints":    4,    "orientation_restraints":    2,
    "angle_restraints":       4,    "angle_restraints_z":     2
  }


In [ ]:
#@markdown A function which modifies an Itp topology object `x`, changing the order of its atom definitions such that hydrogens immediately follow their bonded heavy atoms.
#@markdown ```
#@markdown def interleave_H_in_topology(x): ...
#@markdown ```

from types import SimpleNamespace
S = lambda: SimpleNamespace(atom_obj=dict(), atom_element=dict(), atom_bonded_atoms=dict())

def interleave_H_in_topology(x):
  # Precaching
  element = re.compile(r"(?<=[a-zA-Z])(?=[0-9])")
  cache = dict()
  name = None
  for block, data in zip(x.blocks, x.data):
    if block is None:
      continue
    block_line_ = block.line.replace(" ", "")
    if "[moleculetype]" in block_line_:
      name = next(Itp.only(data)).line[0].strip()
      cache[name] = S()
    elif "[atoms]" in block_line_:
      for t in Itp.only(data):
        l, c = t
        atom_id = int(l[0])
        cache[name].atom_obj[atom_id] = t
        cache[name].atom_element[atom_id] = element.split(l[4])[0].strip()
    elif "[bonds]" in block_line_:
      for l, c in Itp.only(data):
        u, v = int(l[0]), int(l[1])
        for u_, v_ in ((u, v), (v, u)):
          try:
            cache[name].atom_bonded_atoms[u_].append(v_)
          except KeyError:
            cache[name].atom_bonded_atoms[u_] = [v_]

  # Reorder atom entries such that Hs follow the heavy atoms they are bound to
  name = None
  new_data = dict()
  for i, (block, data) in enumerate(zip(x.blocks, x.data)):
    if block is None:
      continue
    block_line_ = block.line.replace(" ", "")
    if "[moleculetype]" in block_line_:
      name = next(Itp.only(data)).line[0].strip()
    elif "[atoms]" in block_line_:
      new_data[i] = list()
      skip = list()
      for t in data:
        l, c = t
        if isinstance(l, list):
          atom_id = int(l[0])
          if atom_id not in skip:
            if cache[name].atom_element[atom_id] != "H": # any heavy atom
              # Put any bonded Hs next to this heavy atom in the new_data list
              new_data[i].append(t)
              if atom_id in cache[name].atom_bonded_atoms:
                for other_id in cache[name].atom_bonded_atoms[atom_id]:
                  if cache[name].atom_element[other_id] == "H":
                    new_data[i].append(cache[name].atom_obj[other_id])
                    skip.append(other_id)
            elif atom_id not in cache[name].atom_bonded_atoms: # a H that isn't bonded to anything?
              new_data[i].append(t)
        else:
          new_data[i].append(t)

  for key, value in new_data.items():
    x.data[key] = value

  # Renumber all atom ids
  name = None
  new_id = None
  atom_id_map = None # old id -> new id

  def str_like(number, template):
    return f"%{len(template)}d" % (number,)

  for block, data in zip(x.blocks, x.data):
    if block is None:
      continue
    block_line_ = block.line.replace(" ", "")
    if "[moleculetype]" in block_line_:
      name = next(Itp.only(data)).line[0].strip()
      new_id = 1
      atom_id_map = dict()
    elif "[atoms]" in block_line_:
      for l, c in Itp.only(data):
        atom_id = int(l[0])
        l[0] = str_like(new_id, l[0]) # l is mutable, and is the same object as x.data[...].t.line
        atom_id_map[atom_id] = new_id
        new_id += 1
    else:
      for key, num_cols in Itp.block_num_atom_cols.items():
        if f"[{key}]" in block_line_:
          for l, c in Itp.only(data):
            if num_cols < 0: # the exclusions block is variable length, all columns are atoms
              num_cols = len(l)
            for i in range(num_cols):
              this_id = int(l[i])
              l[i] = str_like(atom_id_map[this_id], l[i])
          break
  
  # Return a description of the changes made
  return atom_id_map

In [ ]:
#@markdown A class which allows for the parsing and limited editing of **GROMACS ".gro" coordinates files**.
#@markdown ```
#@markdown class Gro:
#@markdown   def __init__(self, filename): ...
#@markdown   ...
#@markdown ```

class Gro:
  def __init__(self, filename):
    self.title = None
    self.info = list() # list of (int res_id, str res_name, str atom_name, int res_name)
    self.pos = list() # list of (float x, float y, float z)
    self.vel = None # list of (float u, float v, float w) or omitted by keeping self.vel = None
    self.width = None # floating point formatting width
    self.box = None
    self.box_explicit_diag = None
    self.box_width = None
    with open(filename) as f:
      self.__parse(f)

  def __parse(self, lines):
    n = None
    for i, l in enumerate(lines):
      l = l.rstrip("\n")
      if i == 0:
        self.title = l
      elif i == 1:
        n = int(l)
      elif 2 <= i < 2 + n:
        s = l[:20]
        d = l[20:].rstrip()
        
        # If this is the first line of data, need to work out the data format being used
        if i == 2:
          num_dots = len([x for x in l if x == "."])
          if num_dots == 3:
            pass
          elif num_dots == 6:
            self.vel = list()
          else:
            raise RuntimeError(f"Error: expected 3 or 6 data columns, but got: {d}")
          if len(d) % num_dots != 0:
            raise RuntimeError(f"Error: expected consistent column width for {num_dots} columns, but got: {d}")
          self.width = len(d) // num_dots
        
        # Populate the data lists
        info = (int(s[:5]), s[5:10].strip(), s[10:15].strip(), int(s[15:20]))
        self.info.append(info)
        pos = tuple(float(d[x:x+self.width]) for x in range(0, 3 * self.width, self.width))
        self.pos.append(pos)
        if self.vel is not None:
          vel = tuple(float(d[x:x+self.width]) for x in range(3 * self.width, 6 * self.width, self.width))
          self.vel.append(vel)
      elif i == 2 + n:
        l = l.rstrip()
        num_dots = len([x for x in l if x == "."])
        if num_dots == 3:
          self.box_explicit_diag = False
        elif num_dots == 9:
          self.box_explicit_diag = True
        else:
          raise RuntimeError(f"Error: expected 3 or 9 box columns, but got: {l}")
        if len(l) % num_dots != 0:
          raise RuntimeError(f"Error: expected consistent column width for box vector, but got: {l}")
        self.box_width = len(l) // num_dots
        self.box = tuple(float(l[x:x+self.box_width]) for x in range(0, len(l), self.box_width))
      else:
        raise RuntimeError(f"Error: expected EOF, but got: {l}")

  def print(self, file=None):
    # Header lines
    print(self.title, file=file)
    print("%5d" % (len(self.info),), file=file)
    # Data lines
    w = self.width
    p = w - 5
    if self.vel is not None:
      v = w - 4
      fmt = "%5d%-5s%5s%5d" + f"%{w}.{p}f" * 3 + f"%{w}.{v}f" * 3
      def fit(x): # valid range is 1-99999, for larger molecules GROMACS allows you to loop around
        return ((x - 1) % 99999) + 1 
      for info, pos, vel in zip(self.info, self.pos, self.vel):
        print(fmt % (fit(info[0]), info[1], info[2], fit(info[3]), *pos, *vel), file=file)
    else:
      fmt = "%5d%-5s%5s%5d" + f"%{w}.{p}f" * 3
      for info, pos in zip(self.info, self.pos):
        print(fmt % (*info, *pos), file=file)
    # Box line
    w = self.box_width
    b = w - 5
    fmt = f"%{w}.{b}f" * (9 if self.box_explicit_diag else 3)
    print(fmt % self.box, file=file)

In [ ]:
#@markdown A function which modifies a Gro coordinates object `y`, changing the order of its atoms to match an interleaved Itp `x`, as per the provided `atom_id_map`.
#@markdown ```
#@markdown def rearrange_coordinates(y, atom_id_map): ...
#@markdown ```

def rearrange_coordinates(y, atom_id_map):
  try:
    data = list(zip(y.info, y.pos, y.vel))
  except TypeError:
    data = list(zip(y.info, y.pos))

  new_data = list()
  for old_id, new_id in atom_id_map.items():
    t = data[old_id - 1]
    t = ((*t[0][:3], new_id), *t[1:])
    new_data.append(t)

  new_data_sep = list(zip(*new_data))

  try:
    y.info, y.pos, y.vel = new_data_sep
  except ValueError:
    y.info, y.pos = new_data_sep

In [ ]:
#@markdown A function which combines multiple "forcefield.itp" / "molecule.prm" topology files, taking care with directive order.
#@markdown ```
#@markdown def combine_itp_files(filenames_in, filename_out): ...
#@markdown ```

def combine_itp_files(filenames_in, filename_out):
  all_parsed_files = [Itp(filename) for filename in filenames_in]

  def index(blocks, key):
    return next((i for i, t in enumerate(blocks) if t == key or hasattr(t, "line") and t.line.replace(" ", "") == f"[{key}]"), None)
    #                      accounts for key is None ^^^

  with open(filename_out, "w") as out:
    for key in Itp.directive_order_hint:
      for x in all_parsed_files:
        print(f"searching for {key}")
        i = index(x.blocks, key)
        print(f"result: {i}")
        if i is not None:
          if x.blocks[i] is not None:
            print(x._str(x.blocks[i]), file=out)
          for t in x.data[i]:
            print(x._str(t), file=out)
          if key == "defaults":
            break # don't print [ defaults ] multiple times

    # For other directives, just put them at the end in the order they appear in their respective .itp files
    for x in all_parsed_files:
      for block, data in zip(x.blocks, x.data):
        if block is not None and "".join(c for c in block.line if c not in "[ ]") not in Itp.directive_order_hint:
          print(x._str(block), file=out)
          for t in data:
            print(x._str(t), file=out)

In [ ]:
%%writefile superpose.py
#!/usr/bin/env python3

#@markdown A script to calculate the translation and rotation matrices when fitting a protein onto another protein.
#@markdown ```
#@markdown superpose.py target.pdb query.pdb
#@markdown -> rms.txt, ro.npy, tr.npy
#@markdown ```

import sys
import numpy as np
from Bio.PDB import Superimposer, PDBParser

par = PDBParser()
target = par.get_structure("target", sys.argv[1])[0]
query = par.get_structure("query", sys.argv[2])[0]

target_atoms = [res["CA"] for ch in target for res in ch if "CA" in res]
query_atoms = [res["CA"] for ch in query for res in ch if "CA" in res]

sup = Superimposer()
sup.set_atoms(target_atoms, query_atoms)

print(f"RMSD: {sup.rms}", file=sys.stderr)

ro, tr = sup.rotran
print("Transformation:", file=sys.stderr)
print(ro, file=sys.stderr)
print(tr, file=sys.stderr)

with open("rms.txt", "w") as o:
  print(sup.rms, file=o)
np.save("ro.npy", ro, allow_pickle=False)
np.save("tr.npy", tr, allow_pickle=False)

In [ ]:
%%writefile insert_molecules_auto_topol
#!/usr/bin/env bash

#@markdown A script which runs `gmx insert-molecules ...` and automatically updates the molecule entries in `topol.top`.
#@markdown ```
#@markdown insert_molecules_auto_topol topol.top LIG TIP3 [args for `gmx insert-molecules`...]
#@markdown ```


topol="$1"
insert_mol="$2"
replace_mol="$3"
shift 3

if [[ ! -s "$topol" || -z "$insert_mol" || -z "$replace_mol" ]]; then
  exit 1
fi

gmx insert-molecules "$@" &> "insert-molecules.log"
ret=$?
if (( $ret != 0 )); then
  cat "insert-molecules.log" >&2
  exit $ret
fi

num_inserted="$(egrep -o "Added [0-9]+ molecules" "insert-molecules.log" | awk '{ print $2 }')"
if (( $num_inserted == 0 )); then
  echo "Error: could not insert molecule ${insert_mol}, no changes made to topology file ${topol}" >&2
  exit 1
fi
num_replaced="$(egrep -o "Replaced [0-9]+ residues" "insert-molecules.log" | awk '{ print $2 }')"

awk \
  -v molins="$insert_mol" \
  -v numins=$num_inserted \
  -v molrep="$replace_mol" \
  -v numrep=$num_replaced \
  '
  !x

  x {
    if ($1 == molins) {
      print $1, $2 + numins
      inserted=1
    }
    else if ($1 == molrep) {
      print $1, $2 - numrep
    }
    else {
      print $0
    }
  }

  $0 ~ /\[ *molecules *\]/ {
    x=1
  }

  END {
    if (!inserted) {
      print molins, numins
    }
  }
  ' \
  "${topol}" \
  > "${topol}.new" \
&& mv "${topol}.new" "${topol}"

#### Processing

**Topology**

In [ ]:
%%bash
#@markdown Get the base **system** (`conf.gro`, `topol.top`, `toppar/`) from the prepared **protein** folder.
#@markdown 
#@markdown Creates the files `SOLV.txt` and optionally `MEMB.txt` containing substructure names corresponding to the CHARMM-GUI indexing of the system.

source "/usr/local/gromacs/bin/GMXRC.bash"

cp protein/gromacs/step*_input.gro "conf.gro"
cp -r protein/gromacs/{index.ndx,topol.top,toppar} .
if [[ ! -s "conf.gro" || ! -s "index.ndx" || ! -s "topol.top" ]]; then
  echo "Error: could not extract protein system" >&2
  exit 1
fi

function gro_residues {
  for f in "$@"; do
    tail -n+3 "${f}" | head -n-1
  done | cut -c6-9 | sort | uniq -c | sort -r | awk '{ print $2 }'
}

if fgrep -q "[ MEMB ]" "index.ndx"; then
  gmx editconf -f "conf.gro" -n "index.ndx" -o "MEMB.gro" <<< "MEMB"
  gro_residues "MEMB.gro" > "MEMB.txt"
  rm "MEMB.gro"
fi

gmx editconf -f "conf.gro" -n "index.ndx" -o "SOLV.gro" <<< "SOLV"
gro_residues "SOLV.gro" > "SOLV.txt"
rm "SOLV.gro"

rm "index.ndx" # needs to be remade after system is edited

In [ ]:
%%bash
#@markdown For each prepared ligand, **generate a GROMACS compatible topology** with (usually) `cgenff_charmm2gmx.py`.

source "/usr/local/gromacs/bin/GMXRC.bash"
eval "$("$START/miniconda3/bin/conda" shell.bash hook)"

:> "add.txt"
:> "restrain.txt"

for d in ligand_*; do
  IFS=_ read type i <<< $d
  if [[ "${i}" == "*" ]]; then
    continue  # no ligand given
  fi

  # Custom ligands, handled with cgenff
  if [[ -s "${type}_${i}/lig/lig.rtf" && -s "${type}_${i}/lig/lig.prm" ]]; then
    
    #
    # Figure out the residue name, like LIG or LIG2
    #

    name_="LIG "
    n=$(find -maxdepth 1 -name "${type}_*" -type d | wc -l)
    if (( $n > 1 )); then
      name_="${name_::$((4 - ${#n}))}${n}" # e.g. LIG + 98 -> LI98
    fi
    name="$(echo $name_)" # strip whitespace
    namell="${name,,}" # lowercase

    #
    # Recover the CGenFF .str file describing the ligand's topology
    #

    mkdir -p "${name}"

    # Expected ".str" file format reverse engineered from cgenff_charmm2gmx_py3_nx2.py
    # (since it doesn't appear to be documented anywhere...)
    {
      echo "* For use with CGenFF version 4.6"
      cat "${type}_${i}/lig/lig.rtf"
      echo "read para"
      cat "${type}_${i}/lig/lig.prm"
    } \
    | sed "s/ lig / ${name_}/g" > "${name}/${name}.str"

    #
    # Convert the topology to GROMACS format
    #

    conda activate "biopython"
    obabel "${type}_${i}/ligandrm.pdb" -O "${name}/${name}.mol2" --title "${name}" --partialcharge "eem2015hn" -xl
    sed -i"" "s/<1> /${name_}/g" "${name}/${name}.mol2"

    pushd "${name}"
    conda activate "charmm2gmx"
    cgenff_charmm2gmx.py "${name}" "${name}.mol2" "${name}.str" "${START}/charmm36.ff"
    popd

    #
    # Include the generated files
    #

    gmx editconf -f "${name}/${namell}_ini.pdb" -o "${name}.gro"
    cp "${name}/${namell}.prm" "toppar/${name}.prm"
    cp "${name}/${namell}.itp" "toppar/${name}.itp"
    
    # Include the complete CHARMM36 forcefield
    if [[ ! -d "toppar/charmm36.ff" ]]; then
      cp -r "${START}/charmm36.ff" "toppar/"
    fi

  # Built in ligands, already part of CHARMM36 
  elif [[ -s "${type}_${i}/gromacs/topol.top" ]]; then
    
    name="$(tail -n1 "${type}_${i}/gromacs/topol.top" | awk '{ print $1 }')"

    gmx editconf -f "${type}_${i}/ligandrm.pdb" -o "${name}.gro"
    cp "${type}_${i}/gromacs/charmm36.itp" "toppar/forcefield_${name}.itp"
    cp "${type}_${i}/gromacs/${name}.itp" "toppar/"

  else
    echo "Error: incompatible molecule: ${type}_${i}" >&2
    exit 1
  fi

  echo "${name}" >> "add.txt"

  # Ligands should have position restraints during equilibration
  echo "${name}" >> "restrain.txt"
done

In [ ]:
#@markdown For each molecule to be added, ensure that **hydrogens are interleaved** such that they appear adjacent to their bonded heavy atoms, in both the topology and coordinates files.
#@markdown
#@markdown (This circumvents an [error in the update group code](https://gromacs.bioexcel.eu/t/gpu-update-giving-error-with-protein-ligand-complex) which prevents the update step from running on the GPU.)

with open("add.txt") as f:
  for l in f:
    l = l.strip()
    x = Itp(f"toppar/{l}.itp")
    atom_id_map = interleave_H_in_topology(x)
    y = Gro(f"{l}.gro")
    rearrange_coordinates(y, atom_id_map)
    with open(f"toppar/{l}.itp", "w") as o:  x.print(file=o)
    with open(f"{l}.gro", "w") as o:  y.print(file=o)

In [ ]:
%%bash
#@markdown Enable **position restraints** for each docked ligand.

source "/usr/local/gromacs/bin/GMXRC.bash"

while read -r name; do
  if fgrep -q "POSRES_FC_LIG" "toppar/${name}.itp"; then
    continue  # already restrained
  fi

  gmx genrestr -f "${name}.gro" -fc 9999 <<< "0"
  {
    echo ""
    echo "#ifdef POSRES"
    tail -n+3 "posre.itp" | sed "s/9999/POSRES_FC_LIG/g"
    echo "#endif"
    echo ""
  } >> "toppar/${name}.itp"
  rm "posre.itp"
done < "restrain.txt"

In [ ]:
#@markdown If a complete `charmm36.ff/` folder for the CHARMM36 forcefield is required because of new topologies in the system, update the `topol.top`.
#@markdown 
#@markdown Otherwise, the `forcefield.itp` files from each merged component must be merged.

forcefield_itps = ["toppar/forcefield.itp"]
try:
  with open("add.txt") as f:
    forcefield_itps += [f"toppar/forcefield_{l.strip()}.itp" for l in f if os.path.isfile(f"toppar/forcefield_{l.strip()}.itp")]
except FileNotFoundError:
  pass

# Complete forcefield
if os.path.isdir("toppar/charmm36.ff"): 
  with open("topol.top") as f, open("topol.top.new", "w") as o:
    for l in f:
      if l.rstrip() == '#include "toppar/forcefield.itp"':
        print('#include "toppar/charmm36.ff/forcefield.itp"', file=o)
      else:
        o.write(l)
  os.rename("topol.top.new", "topol.top")
  
  for itp in forcefield_itps:
    os.remove(itp)

# Partial forcefields to combine
elif len(forcefield_itps) > 1: 
  combine_itp_files(forcefield_itps, "toppar/forcefield.itp")
  
  for itp in forcefield_itps[1:]:
    os.remove(itp)

In [ ]:
#@markdown Add any new `molecule.prm` and `molecule.itp` files to `topol.top`.

prm_itp_files = list()
try:
  with open("add.txt") as f:
    prm_itp_files = [f"toppar/{l.strip()}.{ext}" for l in f for ext in ("itp", "prm") if os.path.isfile(f"toppar/{l.strip()}.{ext}")]
except FileNotFoundError:
  pass

if prm_itp_files:
  prm_files = [f for f in prm_itp_files if f.endswith(".prm")]
  itp_files = [f for f in prm_itp_files if f.endswith(".itp")]

  with open("topol.top") as f, open("topol.top.new", "w") as o:
    l_prev = ""
    for l in f:
      if "forcefield.itp" in l_prev and "forcefield.itp" not in l:
        for prm_file in prm_files:
          print(f'#include "{prm_file}"', file=o)

      if l_prev.startswith("#include") and not l.startswith("#include"):
        for itp_file in itp_files:
          print(f'#include "{itp_file}"', file=o)
      
      o.write(l)
      l_prev = l
  os.rename("topol.top.new", "topol.top")

**Coordinates**

In [ ]:
%%bash
#@markdown Calculate the transformation which maps from the frame of reference in which the protein was inputted to CHARMM-GUI - presumably the same as the **docking frame of reference** - to that of the protein outputted by CHARMM-GUI.

eval "$("$START/miniconda3/bin/conda" shell.bash hook)"
conda activate "biopython"
python3 superpose.py protein/gromacs/step*_input.pdb "protein/step1_pdbreader.pdb"
#                                    ^^^ step3_input for Solution Builder, step5_input for Membrane Builder

rms="$(cat "rms.txt")"
if perl -e "exit !(${rms} > 1.0)"; then
  echo "Error: failed to match CHARMM-GUI protein input coordinates to output coordinates: RMSD = ${rms}" >&2
  exit 1
fi

In [ ]:
#@markdown Transform each docked ligand from the input reference frame to the new reference frame.

import numpy as np

ro = np.load("ro.npy")
tr = np.load("tr.npy")

try:
  with open("restrain.txt") as f:
    for l in f:
      l = l.strip()
      x = Gro(f"{l}.gro")

      pos = np.array(x.pos)
      new_pos = (pos @ ro) + (tr / 10.) # divide by 10 because Angstroms -> nm
      x.pos = [tuple(row) for row in new_pos]

      if x.vel is not None:
        vel = np.array(x.vel)
        new_vel = vel @ ro
        x.vel = [tuple(row) for row in new_vel]

      x.box_explicit_diag = False
      x.box = (0., 0., 0.)

      with open(f"{l}.gro", "w") as o:
        x.print(file=o)
except FileNotFoundError:
  pass

In [ ]:
%%bash
#@markdown **Insert each ligand** into both the coordinates file and the topology.

source "/usr/local/gromacs/bin/GMXRC.bash"

# Back up topol.top
n=$(ls "#topol.top."*"#" 2> /dev/null | wc -l)
cp "topol.top" "#topol.top.${n}#"

while read -r name; do

  # Insert a docked ligand at a fixed location
  echo "0 0 0" > "positions.dat"
  water="$(head -n1 "SOLV.txt")" # SOLV.txt is constructed such that the first line is the water model
  dr=0.0289 # up to an RMSD of 0.5 A
  bash insert_molecules_auto_topol \
    "topol.top" "${name}" "${water}" \
    -f "conf.gro" -ci "${name}.gro" -ip "positions.dat" -rot none -dr $dr $dr $dr -replace "resname ${water}" -try 99 -scale 0.235 -o "out.gro" \
  && mv "out.gro" "conf.gro"

done < "restrain.txt"

In [ ]:
%%bash
#@markdown An **index** is built describing which parts of the system are liquid (solvent + dissolved solute e.g. ions), part of the main complex (protein + cofactors + ligands), and optionally membrane (lipids).

source "/usr/local/gromacs/bin/GMXRC.bash"

gmx make_ndx -f "conf.gro" < /dev/null 2> /dev/null | fgrep "  : " > "index_autodetect.txt"
nr=$(cat "index_autodetect.txt" | wc -l)

#
# Note: groups are named SOLV, MEMB, SOLU, SOLU_MEMB so as to follow
# the CHARMM-GUI convention and maintain compatibility with its
# outputted .mdp configuration files.
#

# Commands to create SOLV (liquid) groups
SOLV_defn="$(cat "SOLV.txt" | tr "\n" "@" | sed 's/^/"/; s/@$/"/; s/@/" | "/g')"
{
  echo "${SOLV_defn}"
  echo "name ${nr} SOLV"
} > "index_commands.txt"
nr=$(($nr + 1))

if [[ ! -s "MEMB.txt" ]]; then

  # The groups SOLU (main), SOLV (liquid) are sufficient to describe the whole system
  {
    echo '! "SOLV"'
    echo "name ${nr} SOLU"
  } >> "index_commands.txt"
  nr=$(($nr + 1))

else

  # Commands to create MEMB (membrane) group
  MEMB_defn="$(cat "MEMB.txt" | tr "\n" "@" | sed 's/^/"/; s/@$/"/; s/@/" | "/g')"
  {
    echo "${MEMB_defn}"
    echo "name ${nr} MEMB"
  } >> "index_commands.txt"
  nr=$(($nr + 1))

  # Commands to create SOLU (everything else) group, which should be the protein and any cofactors, bound ligands, etc.
  {
    echo '! "SOLV" & ! "MEMB"'
    echo "name ${nr} SOLU"
  } >> "index_commands.txt"
  nr=$(($nr + 1))

  # Combine the complex and membrane groups into SOLU_MEMB
  {
    echo '"SOLU" | "MEMB"'
    echo "name ${nr} SOLU_MEMB"
  } >> "index_commands.txt"
  nr=$(($nr + 1))

fi

echo "q" >> "index_commands.txt"

gmx make_ndx -f "conf.gro" -o "index.ndx" < "index_commands.txt"

**Simulation parameters**

In [ ]:
%%bash
#@markdown The `.mdp` configuration files provided by CHARMM-GUI are collated. If a ligand option is present then relevant constraint parameters are added.

# Collate the .mdp configuration files provided by CHARMM-GUI

step0=$(ls -v "protein/gromacs/" | fgrep "minimization.mdp")
cp "protein/gromacs/$step0" "pre_0.mdp" 

i=1
while read -r step; do
  cp "protein/gromacs/$step" "pre_$i.mdp"
  i=$(($i + 1))
done < <(ls -v "protein/gromacs/" | fgrep "equilibration.mdp")


# Add configuration options for the ligand if present

if [[ -s "restrain.txt" ]]; then
  for f in pre_*.mdp; do
    bb="$(head -n1 "$f" | egrep -o "\-DPOSRES_FC_BB=[^ ]+")"
    sed -i"" "s/$bb/$(echo "$bb" | sed 's/_BB/_LIG/') $bb/" "$f"
  done
fi

In [ ]:
%%writefile "production.mdp"
#@markdown Create a .mdp **simulation parameters configuration file** compatible with CHARMM-GUI systems for a **production run**.
integrator  =  md
dt          =  0.002     ; 2 fs
nsteps      =  500000    ; 1 ns simulation time
comm-mode   =  Linear

nstxout  =  10000    ; 50 frames per ns
nstvout  =  10000
; no need to store forces with nstfout as these can be calculated with mdrun -rerun

;
; Canonical settings for CHARMM
; https://pubs.acs.org/doi/10.1021/acs.jctc.5b00935
;

cutoff-scheme  =  Verlet
nstlist        =  40    ; increased for GPU
coulombtype    =  PME
rcoulomb       =  1.2
vdwtype        =  Cut-off
vdw-modifier   =  Force-switch
rvdw-switch    =  1.0
rvdw           =  1.2

tcoupl   =  Nose-Hoover
tc-grps  =  %tc-grps%    ; SOLU    SOLV
tau-t    =  %tau-t%      ; 1.0     1.0
ref-t    =  %ref-t%      ; 303.15  303.15

pcoupl           =  Parrinello-Rahman
pcoupltype       =  %pcoupltype%         ; isotropic
tau-p            =  5.0
compressibility  =  %compressibility%    ; 4.5e-5
ref-p            =  %ref-p%              ; 1.0 

constraints           =  h-bonds
constraint-algorithm  =  LINCS
continuation          =  yes

In [ ]:
%%bash
#@markdown Some settings within the production .mdp file rely on the details of the system (e.g. whether there is a membrane). Edit the file to specify these settings.

sed -i"" "/^#@markdown/d" "production.mdp"

if [[ -s "MEMB.txt" ]]; then

  sed -i"" "
    s/%tc-grps%/SOLU    SOLV    MEMB  /g;
    s/%tau-t%  /1.0     1.0     1.0   /g;
    s/%ref-t%  /303.15  303.15  303.15/g;
    
    s/%pcoupltype%     /semiisotropic /g;
    s/%compressibility%/4.5e-5  4.5e-5/g;
    s/%ref-p%          /1.0     1.0   /g;
  " "production.mdp"

else

  sed -i"" "
    s/%tc-grps%/SOLU    SOLV  /g;
    s/%tau-t%  /1.0     1.0   /g;
    s/%ref-t%  /303.15  303.15/g;
    
    s/%pcoupltype%     /isotropic/g;
    s/%compressibility%/4.5e-5   /g;
    s/%ref-p%          /1.0      /g;
  " "production.mdp"
  
fi

#### Simulation

In [ ]:
%%writefile "run.bash"
output_folder="$1"
#@markdown Create a script to run the simulation.

cp conf.gro restraint.gro

if ! mkdir -p "${output_folder}"; then
  echo "Error: invalid folder: ${output_folder}" >&2
  exit 1
fi

#
# At this point we have:
# conf.gro index.ndx restraint.gro pre_*.mdp topol.top toppar/
#
# The below is a reproduction of the CHARMM-GUI run script, converted to Bash.
#

source "/usr/local/gromacs/bin/GMXRC.bash"
export GMX_MAXCONSTRWARN=-1

echo "Notice: saving output to folder: ${output_folder}"
sleep 1

i=0
while [[ -s "pre_${i}.mdp" ]]; do
  if [[ ! -s "${output_folder}/pre_${i}.gro" ]]; then
    if (( $i == 0 )); then
      gmx grompp -f "pre_${i}.mdp" -o "pre_${i}.tpr" -c "conf.gro" -r "restraint.gro" -p "topol.top" -n "index.ndx" -maxwarn 999
      gmx mdrun -deffnm "pre_${i}"
    else
      prev=$(($i - 1))
      gmx grompp -f "pre_${i}.mdp" -o "pre_${i}.tpr" -c "pre_${prev}.gro" -r "restraint.gro" -p "topol.top" -n "index.ndx" -maxwarn 999
      gmx mdrun -v -stepout 1000 -deffnm "pre_${i}"
    fi
    cp "pre_${i}.gro" "pre_${i}.log" "${output_folder}/"
  fi
  i=$(($i + 1))
done

#
# Now we can save the minimised and equilibrated system
#

i=$(($i - 1))

cp "production.mdp" "${output_folder}/grompp.mdp"
cp "pre_${i}.gro" "${output_folder}/conf.gro"
cp -r "index.ndx" "restraint.gro" "topol.top" "toppar" "${output_folder}/"

In [ ]:
#@markdown Execute the simulation script:
#@markdown 1. Run the **minimisation and equilibration** steps.
#@markdown 2. Save the output to the specified output folder, from which production simulations can be run.

!bash "run.bash" "$output_folder"
!sleep 10

In [ ]:
#@markdown Finally, disconnect the runtime. (This option is ignored if the output folder is not in your Google Drive.)
disconnect = True #@param {type: "boolean"}

if disconnect and output_folder.startswith("/content/drive/MyDrive/"):
  from google.colab import drive, runtime
  drive.flush_and_unmount()
  runtime.unassign()